**kgmyh@naver.com** (월요일 수업 전까지)
이름_머신러닝미니프로젝트.html


- 전처리 
    - 결측치
    - 이상치
    - FE
    - 업무지식

- 모델선택 -> 기본 parameter 생성
        - svm 사용 X
    - ->튜링

- roc - auc가
    - 0.85를 넘는 것을 목표로

## 데이터 셋을 살펴보고 관련 내용 예상

- 값:1 (연체한 적 있음)
- 값:0 (연체한 적 없음)

- RevolvingUtilizationOfUnsecuredLines
    - 전체 운용가능한 돈 대비 현재 운용가능한 돈의 비율
    - 현재 운용할 수 있는 돈의 비율이 높으면 좋긴 하지만 그 대출자가 어느 정도 버는 지를 모르기 때문에 현재 운용할 수 있는 돈을 모를 수도 있을 것 같다
        - 약간 애매하지만 **높을 수록 0값에 가까울 듯**
- Age
    - 나이
    - 나이와 금액 지불 능력이랑은 별개일 것 같다.
        - **관련이 없는 열 일듯**
    
- NumberOfTime30 59DaysPastDueNotWorse
    - 최근 2 년 동안 30 일 ~ 59 일 연체한 횟수
    - 과거 연체한 횟수가 높을 수록 대출자가 다음번에도 연체할 가능성이 높아질 것 같다
        - **높을 수록 1값에 가까울 듯**
        
- DebtRatio


In [1]:
# 데이터 불러오기
import pandas as pd

cols = ['target', 'buget-rate', 'age', 'pastdue30-59', 'debt-ratio', 'monthly-income', 'num-of-creditline', 'pastdue90','num-of-real-estateline', 'pastdue60-80', 'dependents']

df = pd.read_csv("cs_data.csv", index_col=0)
df.columns = cols
df.shape

(150000, 11)

In [2]:
# none 값 지우기
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120269 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   target                  120269 non-null  int64  
 1   buget-rate              120269 non-null  float64
 2   age                     120269 non-null  int64  
 3   pastdue30-59            120269 non-null  int64  
 4   debt-ratio              120269 non-null  float64
 5   monthly-income          120269 non-null  float64
 6   num-of-creditline       120269 non-null  int64  
 7   pastdue90               120269 non-null  int64  
 8   num-of-real-estateline  120269 non-null  int64  
 9   pastdue60-80            120269 non-null  int64  
 10  dependents              120269 non-null  float64
dtypes: float64(4), int64(7)
memory usage: 11.0 MB


In [3]:
df.head()

,target,buget-rate,age,pastdue30-59,debt-ratio,monthly-income,num-of-creditline,pastdue90,num-of-real-estateline,pastdue60-80,dependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
X = df.drop(['target'], axis=1)
y = df['target']
X.info()

<class 'pandas.core.series.Series'>
Int64Index: 120269 entries, 1 to 150000
Series name: target
Non-Null Count   Dtype
--------------   -----
120269 non-null  int64
dtypes: int64(1)
memory usage: 1.8 MB


### 결과 값이 2년내에 대출금 연체할 가능성이 있는 지 여부를 예측하는 것이기 때문에 Clasification을 진행한다
- Decision Tree
- Random Forest
- SVM
- KNN
- Logistic Regression

- import 하기

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

- 데이터 나누기

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
X_train.shape, X_test.shape

((90201, 10), (30068, 10))

In [7]:
from sklearn.metrics import average_precision_score, roc_auc_score
def pmc(y, pred_pos_proba, title=None):
    """
    Average Precision Score와 roc-auc score를 출력
    [parameter]
        y: ndarray - 정답의 label
        pred_pos_proba: ndarray - 모델이 추론한 positive(양성)의 확률
    """
    if title:
        print(title)
    print('roc_auc', roc_auc_score(y, pred_pos_proba))

In [8]:
def print_result(name, model, params=None):
    if params != None:
        gs = GridSearchCV(model, params, cv=4, n_jobs = -1)
        gs.fit(X_train, y_train)
        train_pred = gs.best_estimator_.predict(X_train)
        test_pred = gs.best_estimator_.predict(X_test)
    else:
        model.fit(X_train, y_train)
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)

    pmc(y_train, train_pred, f'{name}의 train set')
    print("-"*50)
    pmc(y_test, test_pred, f'{name}의 test set')
    print("="*100)

### Decision Tree

In [12]:
tree = DecisionTreeClassifier()

params = {
    'max_depth':range(1, 10),
    'max_leaf_nodes':[3, 5, 7, 9]
}
name = "Decision Tree"

print_result(name, tree, params)

Decision Tree의 train set
roc_auc 0.5761259944086895
--------------------------------------------------
Decision Tree의 test set
roc_auc 0.5731350417346964


### Random Forest

In [13]:
rfc = RandomForestClassifier(n_estimators=500)

params = {
    'max_depth':[3, 5]
}
name = "Random Forest"
print_result(name, rfc, params)

Random Forest의 train set
roc_auc 0.5506123354338353
--------------------------------------------------
Random Forest의 test set
roc_auc 0.545400433762373


### SVC  -- 시간이 너무 많이 걸립니다

In [ ]:
svc = Pipeline(steps=[('scaler', StandardScaler()),
                     ('svc', SVC(probability=True))])

name = "SVM"
print_result(name, svc)

### KNN

In [10]:
knn = Pipeline(steps=[('scaler', StandardScaler()),
                     ('knn', KNeighborsClassifier())])

params = {
    'knn__n_neighbors':[3, 5, 7]
}
name = "KNN"
print_result(name, knn, params)

KNN의 train set
roc_auc 0.5838687222192623
--------------------------------------------------
KNN의 test set
roc_auc 0.5537515818940386


### Logistic Regression

In [14]:
lr = Pipeline(steps=[('scaler', StandardScaler()),
                     ('lr', LogisticRegression(solver='liblinear'))])

params = {
    "lr__penalty":['l1', 'l2'],
    "lr__C":[0.01, 0.1, 1, 5, 10]
}
name = "Logistic Regression"
print_result(name, lr, params)

Logistic Regression의 train set
roc_auc 0.5178368977577619
--------------------------------------------------
Logistic Regression의 test set
roc_auc 0.5146711876894746
